In [ ]:
import os
import sys

# Ajouter le répertoire parent au sys.path
sys.path.append(os.path.abspath('..'))

import re
from collections import Counter

import en_core_web_sm
import pandas as pd
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.impute import SimpleImputer
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from spacy.lang.en.stop_words import STOP_WORDS

from training.classes.language_tools import TextProcessing

pd.set_option('display.max_columns', None)

## Introduction et contexte

Ce notebook a pour but de faire une première exploration des données mises à disposition par openfoodfacts pour répondre à notre problème qui est:

quelle alternative à un produit alimentaire pourrait nous être proposé ?

Dans ces alternatives, quels sont les produits qui ne contiennent pas de produit susceptible de provoquer une allergie alimentaire ?

Le volume de données est important, nous allons nous concentrer sur les produits où la liste des ingrédients est renseignés et ou les produits sont vendus en france.

## Chargement des Données

In [2]:
rawdata = None
def extract_raw_data():
    file_path = '../data/en.openfoodfacts.org.products.csv.gz'
    chunk_size = 10000

    filtered_chunks_list = []

    for chunk in pd.read_csv(file_path, chunksize=chunk_size, compression='gzip', sep='\t', engine='python', quoting=3):
        filtered_chunks = chunk[(chunk['countries_tags'] == 'en:france') & (chunk['ingredients_tags'].notna())]
        filtered_chunks_list.append(filtered_chunks)

    if filtered_chunks_list:
        rawdata = pd.concat(filtered_chunks_list, axis=0)
    return rawdata

rawdata = extract_raw_data()
# rawdata = pd.read_csv('filtered_dataset_openfoodfacts_raw_en.csv')
display(rawdata)

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,last_modified_by,last_updated_t,last_updated_datetime,product_name,abbreviated_product_name,generic_name,quantity,packaging,packaging_tags,packaging_en,packaging_text,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,origins_en,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,ingredients_tags,ingredients_analysis_tags,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,serving_quantity,no_nutrition_data,additives_n,additives,additives_tags,additives_en,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,food_groups,food_groups_tags,food_groups_en,states,states_tags,states_en,brand_owner,ecoscore_score,ecoscore_grade,nutrient_levels_tags,product_quantity,owner,data_quality_errors_tags,unique_scans_n,popularity_tags,completeness,last_image_t,last_image_datetime,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,butyric-acid_100g,caproic-acid_100g,caprylic-acid_100g,capric-acid_100g,lauric-acid_100g,myristic-acid_100g,palmitic-acid_100g,stearic-acid_100g,arachidic-acid_100g,behenic-acid_100g,lignoceric-acid_100g,cerotic-acid_100g,montanic-acid_100g,melissic-acid_100g,unsaturated-fat_100g,monounsaturated-fat_100g,omega-9-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,omega-6-fat_100g,alpha-linolenic-acid_100g,eicosapentaenoic-acid_100g,docosahexaenoic-acid_100g,linoleic-acid_100g,arachidonic-acid_100g,gamma-linolenic-acid_100g,dihomo-gamma-linolenic-acid_100g,oleic-acid_100g,elaidic-acid_100g,gondoic-acid_100g,mead-acid_100g,erucic-acid_100g,nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,added-sugars_100g,sucrose_100g,glucose_100g,fructose_100g,lactose_100g,maltose_100g,maltodextrins_100g,starch_100g,polyols_100g,erythritol_100g,fiber_100g,soluble-fiber_100g,insoluble-fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,added-salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g,sulphate_100g,nitrate_100g,acidity_100g
4,5,http://world-en.openfoodfacts.org/product/0000...,touchette,1605337720,2020-11-14T07:08:40Z,1729432954,2024-10-20T14:02:34Z,roboto-app,1.729433e+09,2024-10-20T14:02:34Z,Bio inulin,NaN,NaN,550g,Glass,en:glass,Glass,NaN,EWL,ewl,Gemüse,"en:plant-based-foods-and-beverages,en:plant-ba...","Plant-based foods and beverages,Plant-based fo...",NaN,NaN,NaN,bénivay-ollon,benivay-ollon,en:no-lactose,en:no-lactose,No lactose,13089c,13089c,NaN,NaN,NaN,France,NaN,France,en:france,France,"Jus et purée d'abricots (50%), eau, sucre.","en:apricot-juice-and-puree,en:fruit,en:prunus-...","en:palm-oil-free,en:vegan,en:vegetarian",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

## Inspection des Données Brutes

In [3]:
# Basic stats
print("Taille du dataset:")
print("Number of rows : {}".format(rawdata.shape[0]))
print("Number of columns : {}".format(rawdata.shape[1]))
print()
print("---------------------------")
print()

print("Basics infos:")
print()
display(rawdata.info())
print()
print("---------------------------")
print()

print("Basics statistics: ")
print()
data_desc = rawdata.describe(include='all')
display(data_desc)
print()
print("---------------------------")
print()

print("Unique elements by feature: ")
print()
display(rawdata.nunique().sort_values())
print()
print("---------------------------")
print()

print("Percentage of missing values: ")
print()
null_percentage = 100 * rawdata.isnull().mean()
null_percentage_df = null_percentage.to_frame(name='Null Percentage').T
display(null_percentage_df) 

Taille du dataset:
Number of rows : 262446
Number of columns : 206

---------------------------

Basics infos:

<class 'pandas.core.frame.DataFrame'>
Index: 262446 entries, 4 to 3498839
Columns: 206 entries, code to acidity_100g
dtypes: float64(131), int64(2), object(73)
memory usage: 414.5+ MB


None


---------------------------

Basics statistics: 



,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,last_modified_by,last_updated_t,last_updated_datetime,product_name,abbreviated_product_name,generic_name,quantity,packaging,packaging_tags,packaging_en,packaging_text,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,origins_en,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,ingredients_tags,ingredients_analysis_tags,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,serving_quantity,no_nutrition_data,additives_n,additives,additives_tags,additives_en,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,food_groups,food_groups_tags,food_groups_en,states,states_tags,states_en,brand_owner,ecoscore_score,ecoscore_grade,nutrient_levels_tags,product_quantity,owner,data_quality_errors_tags,unique_scans_n,popularity_tags,completeness,last_image_t,last_image_datetime,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,butyric-acid_100g,caproic-acid_100g,caprylic-acid_100g,capric-acid_100g,lauric-acid_100g,myristic-acid_100g,palmitic-acid_100g,stearic-acid_100g,arachidic-acid_100g,behenic-acid_100g,lignoceric-acid_100g,cerotic-acid_100g,montanic-acid_100g,melissic-acid_100g,unsaturated-fat_100g,monounsaturated-fat_100g,omega-9-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,omega-6-fat_100g,alpha-linolenic-acid_100g,eicosapentaenoic-acid_100g,docosahexaenoic-acid_100g,linoleic-acid_100g,arachidonic-acid_100g,gamma-linolenic-acid_100g,dihomo-gamma-linolenic-acid_100g,oleic-acid_100g,elaidic-acid_100g,gondoic-acid_100g,mead-acid_100g,erucic-acid_100g,nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,added-sugars_100g,sucrose_100g,glucose_100g,fructose_100g,lactose_100g,maltose_100g,maltodextrins_100g,starch_100g,polyols_100g,erythritol_100g,fiber_100g,soluble-fiber_100g,insoluble-fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,added-salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g,sulphate_100g,nitrate_100g,acidity_100g
count,262446,262446,262445,2.624460e+05,262446,2.624460e+05,262446,255918,2.623840e+05,262384,259542,14365,72756,193203,131260,131260,131260,12133,247022,247013,224425,224425,224425,50696,50660,50659,62115,62091,162683,162683,162683,62206,62195,48498,0.0,51890,77227,113823,262446,262446,262446,262445,262446,262444,101898,0.0,60167,78640,77682,65262,6.330200e+04,13393,262446.000000,2,134324,134324,193651.000000,262428,226888.000000,262445,262446,202214,202214,202214,262446,262446,262446,3884,161513.000000,262438,195942,191720.000000,50955,8612,191012.000000,187639,262446.000000,2.561790e+05,256179,224425,224425,243009,243009,228118,228118,185463


---------------------------

Unique elements by feature: 



allergens_en                0
water-hardness_100g         0
cities                      0
added-salt_100g             0
elaidic-acid_100g           0
                        ...  
last_image_t           254506
created_datetime       257802
created_t              257802
code                   261662
url                    261668
Length: 206, dtype: int64


---------------------------

Percentage of missing values: 



,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,last_modified_by,last_updated_t,last_updated_datetime,product_name,abbreviated_product_name,generic_name,quantity,packaging,packaging_tags,packaging_en,packaging_text,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,origins_en,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,ingredients_tags,ingredients_analysis_tags,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,serving_quantity,no_nutrition_data,additives_n,additives,additives_tags,additives_en,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,food_groups,food_groups_tags,food_groups_en,states,states_tags,states_en,brand_owner,ecoscore_score,ecoscore_grade,nutrient_levels_tags,product_quantity,owner,data_quality_errors_tags,unique_scans_n,popularity_tags,completeness,last_image_t,last_image_datetime,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,butyric-acid_100g,caproic-acid_100g,caprylic-acid_100g,capric-acid_100g,lauric-acid_100g,myristic-acid_100g,palmitic-acid_100g,stearic-acid_100g,arachidic-acid_100g,behenic-acid_100g,lignoceric-acid_100g,cerotic-acid_100g,montanic-acid_100g,melissic-acid_100g,unsaturated-fat_100g,monounsaturated-fat_100g,omega-9-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,omega-6-fat_100g,alpha-linolenic-acid_100g,eicosapentaenoic-acid_100g,docosahexaenoic-acid_100g,linoleic-acid_100g,arachidonic-acid_100g,gamma-linolenic-acid_100g,dihomo-gamma-linolenic-acid_100g,oleic-acid_100g,elaidic-acid_100g,gondoic-acid_100g,mead-acid_100g,erucic-acid_100g,nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,added-sugars_100g,sucrose_100g,glucose_100g,fructose_100g,lactose_100g,maltose_100g,maltodextrins_100g,starch_100g,polyols_100g,erythritol_100g,fiber_100g,soluble-fiber_100g,insoluble-fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,added-salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g,sulphate_100g,nitrate_100g,acidity_100g
Null Percentage,0.0,0.0,0.000381,0.0,0.0,0.0,0.0,2.487369,0.023624,0.023624,1.106513,94.526493,72.277726,26.383713,49.985902,49.985902,49.985902,95.376954,5.877019,5.880448,14.487171,14.487171,14.487171,80.683264,80.696981,80.697363,76.332274,76.341419,38.012772,38.012772,38.012772,76.2976,76.301792,81.52077,100.0,80.228314,70.574137,56.629935,0.0,0.0,0.0,0.000381,0.0,0.000762,61.173727,100.0,77.074522,70.035741,70.400768,75.13317,75.879991,94.896855,0.0,99.999238,48.818424,48.818424,26.213011,0.006859,13.548692,0.000381,0.0,22.950245,22.950245,22.950245,0.0,0.0,0.0,98.520077,38.458578,0.003048,25.34007,26.948782,80.584577,96.718563,27.218552,2

**Notes :**

Il y a beaucoup de données manquantes.

Nous allons dans un premier temps nous concentrer sur les données des produits vendus en France.

## Analyse Exploratoire des Données

In [4]:
# Find different states
unique_values = set()
rawdata['states_tags'].str.split(',').apply(unique_values.update)

states_df = pd.DataFrame(data=unique_values, columns=['states'])
display(states_df)

,states
0,en:photos-uploaded
1,en:characteristics-completed
2,en:packaging-photo-to-be-selected
3,en:ingredients-photo-to-be-selected
4,en:characteristics-to-be-completed
5,en:categories-to-be-completed
6,en:ingredients-completed
7,en:front-photo-to-be-selected
8,en:expiration-date-completed
9,en:categories-completed


In [5]:
# Vérifier que la colonne 'state' existe
if 'state' in rawdata.columns:
    # Compter les occurrences de chaque état
    states_count = rawdata['state'].value_counts().reset_index()
    states_count.columns = ['state', 'count']
    
    # Afficher le tableau
    print(states_count)
else:
    print("La colonne 'state' n'existe pas dans le dataset.")

La colonne 'state' n'existe pas dans le dataset.


**Notes :**

Il y a beaucoup de colonnes, nous allons sélectionner celles qui potentiellement pourront aider nos futurs modèles à apprendre et à répondre à notre problématique.

In [6]:
columns_to_keep = [
    'code',
    'url',
    'created_datetime',
    'last_modified_datetime',
    'product_name',
    'packaging_tags',
    'categories_tags',
    'ingredients_tags',
    'ingredients_analysis_tags',
    'allergens',
    'traces_tags',
    'additives_n',
    'additives_tags',
    'nutriscore_grade',
    'food_groups_tags',
    'states_tags',
    'ecoscore_score',
    'ecoscore_grade',
    'nutrient_levels_tags',
    'popularity_tags',
    'main_category',
    'image_url',
    'image_small_url',
    'image_ingredients_url',
    'image_ingredients_small_url',
    'image_nutrition_url',
    'image_nutrition_small_url',
    'energy-kcal_100g',
    'energy_100g',
    'fat_100g',
    'saturated-fat_100g',
    'cholesterol_100g',
    'sugars_100g',
    'proteins_100g',
    'salt_100g',
    'fruits-vegetables-nuts-estimate-from-ingredients_100g'
]

data = rawdata[columns_to_keep]
display(data)

,code,url,created_datetime,last_modified_datetime,product_name,packaging_tags,categories_tags,ingredients_tags,ingredients_analysis_tags,allergens,traces_tags,additives_n,additives_tags,nutriscore_grade,food_groups_tags,states_tags,ecoscore_score,ecoscore_grade,nutrient_levels_tags,popularity_tags,main_category,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kcal_100g,energy_100g,fat_100g,saturated-fat_100g,cholesterol_100g,sugars_100g,proteins_100g,salt_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g
4,5,http://world-en.openfoodfacts.org/product/0000...,2020-11-14T07:08:40Z,2024-10-20T14:02:34Z,Bio inulin,en:glass,"en:plant-based-foods-and-beverages,en:plant-ba...","en:apricot-juice-and-puree,en:fruit,en:prunus-...","en:palm-oil-free,en:vegan,en:vegetarian",NaN,NaN,0.0,NaN,a,"en:fruits-and-vegetables,en:vegetables","en:to-be-completed,en:nutrition-facts-complete...",NaN,unknown,"en:fat-in-low-quantity,en:saturated-fat-in-low...",NaN,en:vegetables,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,208.0,840.0,0.0,0.0,NaN,8.0,0.0,0.23,50.000000
9,10,http://world-en.openfoodfacts.org/product/0000...,2016-10-20T07:19:01Z,2024-10-04T09:38:47Z,Madeleines nature,"en:plastic,en:cardboard,fr:boite-en-carton,fr:...","en:snacks,en:desserts,en:sweet-snacks,en:biscu...","en:wheat-flour,en:cereal,en:flour,en:wheat,en:...","en:palm-oil-free,en:non-vegan,en:vegetarian-st...","en:eggs,en:gluten,en:milk","en:nuts,en:soybeans",3.0,"en:e331,en:e422,en:e503",d,"en:sugary-snacks,en:biscuits-and-cakes","en:to-be-completed,en:nutrition-facts-complete...",53.0,c,"en:fat-in-high-quantity,en:saturated-fat-in-mo...","top-50000-scans-2019,top-100000-scans-2019,at-...",en:plain-madeleines,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,442.0,1852.0,22.0,2.6,NaN,25.0,6.4,0.53,22.666667
14,15,http://world-en.openfoodfacts.org/product/0000...,2018-04-15T16:43:14Z,2024-10-04T09:38:50Z,Madeleines ChocoLait,"en:plastic,en:cardboard","en:snacks,en:sweet-snacks,en:biscuits-and-cake...","en:wheat-flour,en:cereal,en:flour,en:wheat,en:...","en:palm-oil-free,en:non-vegan,en:maybe-vegetarian",NaN,en:nuts,5.0,"en:e322,en:e322i,en:e331,en:e422,en:e500,en:e503",d,"en:sugary-snacks,en:biscuits-and-cakes","en:to-be-completed,en:nutrition-facts-complete...",32.0,d,"en:fat-in-high-quantity,en:saturated-fat-in-hi...","top-50000-scans-2019,top-100000-scans-2019,at-...",en:chocolate-madeleines,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,460.0,1926.0,24.0,6.0,NaN,31.0,6.4,0.48,16.250000
18,20,http://world-en.openfoodfacts.org/product/0000...,2018-09-14T13:14:06Z,2024-10-04T09:38:53Z,Madeleines Choco Noir,fr:1-boite-en-carton-a-recycler-50-sachets-ind...,"en:snacks,en:sweet-snacks,en:biscuits-and-cake...","en:wheat-flour,en:cereal,en:flour,en:wheat,en:...","en:palm-oil-free,en:non-vegan,en:vegetarian-st...","en:eggs,en:gluten,en:milk,en:soybeans",en:nuts,5.0,"en:e322,en:e331,en:e422,en:e500,en:e503",d,"en:sugary-snacks,en:biscuits-and-cakes","en:to-be-checked,en:complete,en:nutrition-fact...",39.0,d,"en:fat-in-high-quantity,en:saturated-fat-in-hi...","top-50000-scans-2019,top-100000-scans-2019,at-...",en:chocolate-madeleines,https://images.openfoo

In [7]:
# Liste des états cibles
target_states = ["en:completed", "en:origins-completed", "en:ingredients-completed"]

# Construire une expression régulière pour les états
pattern = '|'.join(target_states)

# Compter les lignes contenant au moins un des états cibles
count = data['states_tags'].str.contains(pattern).sum()

print("Nombre de lignes avec les états spécifiés:", count)


Nombre de lignes avec les états spécifiés: 258361


In [8]:
data = data[data['states_tags'].str.contains(pattern)]

In [9]:
count_data_allergen = data['allergens'].notna().sum()
print("Nombre de lignes avec des allergens:", count_data_allergen)

Nombre de lignes avec des allergens: 100665


**Notes:**

Nous allons remplacer les valeurs de la colonne allergens non renseignées par la valeur `en:none`.

In [10]:
data.loc[:, 'allergens'] = data['allergens'].astype(str)
data['allergens'] = data['allergens'].str.lower()
data['allergens'] = data['allergens'].apply(lambda x: 'en:none' if x == 'nan' else x)
data['allergens'].value_counts()

allergens
en:none                                                                       158457
en:milk                                                                        23667
en:gluten                                                                      12388
en:gluten,en:milk                                                               5871
en:eggs,en:gluten,en:milk                                                       5612
                                                                               ...  
en:eggs,fr:omelette                                                                1
en:celery,en:eggs,en:gluten,en:milk,fr:curry                                       1
en:celery,en:crustaceans,en:eggs,en:gluten,en:milk,en:molluscs,en:soybeans         1
en:eggs,en:gluten,en:milk,fr:thon albacore 45 %,fr:colin d'alaska 30 %             1
en:eggs,en:gluten,en:milk,en:nuts,en:peanuts,fr:biscuit spéculoos                  1
Name: count, Length: 3144, dtype: int64

In [11]:
# Divise chaque entrée par ',' et compile tous les allergènes en une seule liste
allergens_list = data["allergens"].fillna('').str.split(',').sum()

# Compte les occurrences de chaque allergène
allergen_counts = Counter(allergens_list)

# Convertir le résultat en DataFrame
allergens_df = pd.DataFrame(allergen_counts.items(), columns=['allergen', 'count']).sort_values(by='count', ascending=False)
display(allergens_df)

,allergen,count
0,en:none,158511
3,en:milk,56302
2,en:gluten,44189
1,en:eggs,20899
4,en:soybeans,15370
...,...,...
725,fr:disufite de sodium,1
726,fr:lait entier crème,1
727,fr:cacahete,1
728,fr:emmental français,1


**Notes :**

On peu se fier seulement aux allergènes contenant plus de 100 produits dans la base de données.

Nous pourronts mettre les autres dans une autre catégories nommée `other`.

In [12]:
display(allergens_df[allergens_df['count'] > 100])

,allergen,count
0,en:none,158511
3,en:milk,56302
2,en:gluten,44189
1,en:eggs,20899
4,en:soybeans,15370
6,en:nuts,10530
8,en:fish,8302
10,en:sulphur-dioxide-and-sulphites,7400
7,en:mustard,5597
5,en:celery,4356


**Notes:**

Nous allons considérer que la valeur vide correspond à pas d'allergène : `en:none`.

Nous allons mettre les allergènes qui ne sont pas dans cette liste dans une catégorie `other`.

Nous pouvons fusionner les valeurs `fr:Non` et `en:none` et traduire `fr:avoine` en `en:oats`.

In [13]:
# Liste des allergènes à conserver (ceux dont le count est supérieur à 100)
allergens_to_keep = allergens_df[allergens_df['count'] > 100]['allergen'].tolist()
allergens_to_keep.append('en:none') # Ajoute l'entrée none

data['allergens'] = data['allergens'].apply(lambda x: 'other' if x not in allergens_to_keep  else x)
data['allergens'] = data['allergens'].apply(lambda x: 'en:oats' if x == 'fr:avoine' else x)
data['allergens'].value_counts()

display(data)

,code,url,created_datetime,last_modified_datetime,product_name,packaging_tags,categories_tags,ingredients_tags,ingredients_analysis_tags,allergens,traces_tags,additives_n,additives_tags,nutriscore_grade,food_groups_tags,states_tags,ecoscore_score,ecoscore_grade,nutrient_levels_tags,popularity_tags,main_category,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kcal_100g,energy_100g,fat_100g,saturated-fat_100g,cholesterol_100g,sugars_100g,proteins_100g,salt_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g
4,5,http://world-en.openfoodfacts.org/product/0000...,2020-11-14T07:08:40Z,2024-10-20T14:02:34Z,Bio inulin,en:glass,"en:plant-based-foods-and-beverages,en:plant-ba...","en:apricot-juice-and-puree,en:fruit,en:prunus-...","en:palm-oil-free,en:vegan,en:vegetarian",en:none,NaN,0.0,NaN,a,"en:fruits-and-vegetables,en:vegetables","en:to-be-completed,en:nutrition-facts-complete...",NaN,unknown,"en:fat-in-low-quantity,en:saturated-fat-in-low...",NaN,en:vegetables,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,208.0,840.0,0.0,0.0,NaN,8.0,0.0,0.23,50.000000
9,10,http://world-en.openfoodfacts.org/product/0000...,2016-10-20T07:19:01Z,2024-10-04T09:38:47Z,Madeleines nature,"en:plastic,en:cardboard,fr:boite-en-carton,fr:...","en:snacks,en:desserts,en:sweet-snacks,en:biscu...","en:wheat-flour,en:cereal,en:flour,en:wheat,en:...","en:palm-oil-free,en:non-vegan,en:vegetarian-st...",other,"en:nuts,en:soybeans",3.0,"en:e331,en:e422,en:e503",d,"en:sugary-snacks,en:biscuits-and-cakes","en:to-be-completed,en:nutrition-facts-complete...",53.0,c,"en:fat-in-high-quantity,en:saturated-fat-in-mo...","top-50000-scans-2019,top-100000-scans-2019,at-...",en:plain-madeleines,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,442.0,1852.0,22.0,2.6,NaN,25.0,6.4,0.53,22.666667
14,15,http://world-en.openfoodfacts.org/product/0000...,2018-04-15T16:43:14Z,2024-10-04T09:38:50Z,Madeleines ChocoLait,"en:plastic,en:cardboard","en:snacks,en:sweet-snacks,en:biscuits-and-cake...","en:wheat-flour,en:cereal,en:flour,en:wheat,en:...","en:palm-oil-free,en:non-vegan,en:maybe-vegetarian",en:none,en:nuts,5.0,"en:e322,en:e322i,en:e331,en:e422,en:e500,en:e503",d,"en:sugary-snacks,en:biscuits-and-cakes","en:to-be-completed,en:nutrition-facts-complete...",32.0,d,"en:fat-in-high-quantity,en:saturated-fat-in-hi...","top-50000-scans-2019,top-100000-scans-2019,at-...",en:chocolate-madeleines,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,460.0,1926.0,24.0,6.0,NaN,31.0,6.4,0.48,16.250000
18,20,http://world-en.openfoodfacts.org/product/0000...,2018-09-14T13:14:06Z,2024-10-04T09:38:53Z,Madeleines Choco Noir,fr:1-boite-en-carton-a-recycler-50-sachets-ind...,"en:snacks,en:sweet-snacks,en:biscuits-and-cake...","en:wheat-flour,en:cereal,en:flour,en:wheat,en:...","en:palm-oil-free,en:non-vegan,en:vegetarian-st...",other,en:nuts,5.0,"en:e322,en:e331,en:e422,en:e500,en:e503",d,"en:sugary-snacks,en:biscuits-and-cakes","en:to-be-checked,en:complete,en:nutrition-fact...",39.0,d,"en:fat-in-high-quantity,en:saturated-fat-in-hi...","top-50000-scans-2019,top-100000-scans-2019,at-...",en:chocolate-madeleines,https://images.openfoodfacts.org/images/produc...,https://images.openf

In [14]:
# Séparer les allergènes par ",", et convertir en une liste
allergens_series = data['allergens'].str.split(',').explode().str.strip()
# Compter les occurrences de chaque allergène
allergens_counts = allergens_series.value_counts().reset_index()
allergens_counts.columns = ['allergen', 'count']

# Créer le graphique
fig = px.bar(allergens_counts, x='allergen', y='count', 
             title="Répartition des allergènes dans le dataset",
             labels={'allergen': 'Allergènes', 'count': 'Nombre d\'occurrences'},
             height=500)

fig.show()

Maintenant nous allons nous focaliser sur les traces d'allergènes.

In [15]:
data.loc[:, 'traces_tags'] = data['traces_tags'].astype(str).str.lower()

# Divise chaque entrée par ',' et compile tous les traces d'allergènes en une seule liste
traces_allergens_list = data["traces_tags"].fillna('').str.split(',').sum()

# Compte les occurrences de chaque traces d'allergène
traces_allergen_counts = Counter(traces_allergens_list)

# Convertir le résultat en DataFrame
traces_allergens_df = pd.DataFrame(traces_allergen_counts.items(), columns=['allergen', 'count']).sort_values(by='count', ascending=False)
display(allergens_df)

,allergen,count
0,en:none,158511
3,en:milk,56302
2,en:gluten,44189
1,en:eggs,20899
4,en:soybeans,15370
...,...,...
725,fr:disufite de sodium,1
726,fr:lait entier crème,1
727,fr:cacahete,1
728,fr:emmental français,1


In [16]:
display(traces_allergens_df[traces_allergens_df['count'] > 20])

,allergen,count
0,nan,180752
1,en:nuts,41442
2,en:soybeans,28009
11,en:milk,25267
4,en:gluten,21943
6,en:eggs,21711
5,en:sesame-seeds,17072
20,en:mustard,13961
7,en:celery,13398
17,en:peanuts,10851


In [17]:
# Séparer les allergènes par ",", et convertir en une liste
data.loc[:, 'traces_tags'] = data['traces_tags'].astype(str)

traces_allergens_to_keep = traces_allergens_df[traces_allergens_df['count'] > 20]['allergen'].tolist()

data['traces_tags'] = data['traces_tags'].apply(lambda x: 'en:none' if (x == 'nan')  else x)
data['traces_tags'] = data['traces_tags'].apply(lambda x: 'other' if x not in traces_allergens_to_keep  else x)

traces_allergens_series = data['traces_tags'].str.split(',').explode().str.strip()
traces_allergens_counts = traces_allergens_series.value_counts().reset_index()
traces_allergens_counts.columns = ['traces_tags', 'count']

# Créer le graphique
fig = px.bar(traces_allergens_counts, x='traces_tags', y='count', 
             title="Répartition des traces d'allergènes dans le dataset",
             labels={'traces_tags': 'Traces allergènes', 'count': 'Nombre d\'occurrences'},
             height=500)

fig.show()

### Sélections et nettoyage des données

In [18]:
data.describe(include='all')

,code,url,created_datetime,last_modified_datetime,product_name,packaging_tags,categories_tags,ingredients_tags,ingredients_analysis_tags,allergens,traces_tags,additives_n,additives_tags,nutriscore_grade,food_groups_tags,states_tags,ecoscore_score,ecoscore_grade,nutrient_levels_tags,popularity_tags,main_category,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kcal_100g,energy_100g,fat_100g,saturated-fat_100g,cholesterol_100g,sugars_100g,proteins_100g,salt_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g
count,258361,258361,258361,258361,255537,130790,222035,258361,258359,258361,258361,258361.000000,131893,258345,200085,258361,159886.000000,258353,193750,185127,222035,239625,239625,224313,224313,183187,183187,2.139710e+05,2.318000e+05,231549.000000,229837.000000,637.000000,230437.000000,231393.000000,228250.000000,258359.000000
unique,257585,257591,253809,226334,197147,18477,49675,196813,37,19,33,NaN,33170,7,45,2159,NaN,9,225,86223,17077,238878,238878,223596,223596,182612,182612,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,3250390663201,http://world-en.openfoodfacts.org/product/3254...,2021-04-20T10:35:38Z,2023-10-26T17:32:26Z,Comté,"en:plastic,en:bag",en:beverages,"en:superior-quality-durum-wheat-semolina,en:ce...","en:palm-oil-free,en:vegan,en:vegetarian",en:none,en:none,NaN,"en:e322,en:e322i",unknown,"en:sugary-snacks,en:biscuits-and-cakes","en:to-be-completed,en:nutrition-facts-complete...",NaN,unknown,"en:fat-in-low-quantity,en:saturated-fat-in-low...","bottom-25-percent-scans-2019,bottom-20-percent...",en:groceries,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,2,2,13,23,204,7450,1681,784,45406,158457,181708,NaN,6665,56101,17355,20867,NaN,94324,22137,6150,8684,2,2,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.529871,NaN,NaN,NaN,NaN,55.388873,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.636413e+10,2.566868e+11,13.531768,5.250109,0.128933,13.672234,8.337464,1.148384,19.084162
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.332687,NaN,NaN,NaN,NaN,28.206574,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.069806e+13,1.235834e+14,17.007197,7.945012,1.446770,22.894757,9.162811,7.355613,48.318683
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,-30.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,-0.100000,0.000000,0.000000,-65.887256
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,35.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.110000e+02,4.610000e+02,1.100000,0.260000,0.000000,0.800000,1.700000,0.050000,0.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,56.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.520000e+02,1.040000e+03,7.200000,2.000000,0.000000,3.600000,6.000000,0.500000,0.044759
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,NaN,NaN,NaN,NaN,76.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.930000e+02,1.636000e+03,21.000000,7.200000,0.016700,19.000000,12.000000,1.200000,22.525240


In [19]:
data[data['additives_n'] == 1.0].head()

,code,url,created_datetime,last_modified_datetime,product_name,packaging_tags,categories_tags,ingredients_tags,ingredients_analysis_tags,allergens,traces_tags,additives_n,additives_tags,nutriscore_grade,food_groups_tags,states_tags,ecoscore_score,ecoscore_grade,nutrient_levels_tags,popularity_tags,main_category,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kcal_100g,energy_100g,fat_100g,saturated-fat_100g,cholesterol_100g,sugars_100g,proteins_100g,salt_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g
49,51,http://world-en.openfoodfacts.org/product/0000...,2016-12-01T19:59:24Z,2024-10-11T08:26:38Z,Fondants Citron,"fr:boite-carton,fr:sachet-plastique","en:plant-based-foods-and-beverages,en:plant-ba...","en:blackberry,en:fruit,en:berries,en:cane-suga...","en:palm-oil-free,en:vegan-status-unknown,en:ve...",other,other,1.0,en:e440,c,"en:sugary-snacks,en:sweets","en:to-be-completed,en:nutrition-facts-complete...",64.0,b,"en:fat-in-low-quantity,en:saturated-fat-in-low...","top-50000-scans-2019,top-100000-scans-2019,at-...",en:jams,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,249.0,1056.0,0.30,0.05,NaN,59.00,0.5,0.050,50.0
120,131,http://world-en.openfoodfacts.org/product/0000...,2020-06-30T12:29:48Z,2024-10-29T14:06:01Z,Confiture de fraise mara des bois,NaN,"en:plant-based-foods-and-beverages,en:plant-ba...","en:e968,en:monk-fruit-extract,en:sweetener","en:palm-oil-free,en:vegan-status-unknown,en:ve...",en:none,en:none,1.0,en:e968,c,"en:sugary-snacks,en:sweets","en:to-be-completed,en:nutrition-facts-complete...",60.0,b,"en:fat-in-low-quantity,en:saturated-fat-in-low...",NaN,en:strawberry-jams,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,NaN,NaN,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,0.0,0.0,0.00,0.00,NaN,725.00,0.0,0.000,0.0
170,184,http://world-en.openfoodfacts.org/product/0000...,2023-02-25T11:51:38Z,2024-10-04T09:45:42Z,Whey Protein Vanilla,NaN,en:dietary-supplements,"fr:ingredients,fr:tunique,fr:extrait-de-fruits...","en:palm-oil-content-unknown,en:vegan-status-un...",other,en:none,1.0,en:e464,b,NaN,"en:to-be-completed,en:nutrition-facts-complete...",NaN,unknown,"en:fat-in-moderate-quantity,en:saturated-fat-i...",NaN,en:dietary-supplements,NaN,NaN,NaN,NaN,NaN,NaN,377.0,1580.0,4.33,1.33,NaN,3.33,80.0,0.233,0.0
260,289,http://world-en.openfoodfacts.org/product/0000...,2019-09-14T10:42:53Z,2024-10-04T09:40:01Z,Passiflore BIO,NaN,en:dietary-supplements,fr:complement-alimentair-ingredients-pour-3-co...,"en:palm-oil-content-unknown,en:vegan-status-un...",en:none,en:none,1.0,"en:e553,en:e553b",unknown,NaN,"en:to-be-completed,en:nutrition-facts-complete...",NaN,unknown,NaN,"bottom-25-percent-scans-2019,bottom-20-percent...",en:dietary-supplements,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
402,476,http://world-en.openfoodfacts.org/product/0000...,2019-08-19T21:04:21Z,2024-10-11T14:12:41Z,Encore +,flacon,fr:lubrifiant-feminin,"fr:cyclopentasiloxane,en:e900,fr:paraben-free,...","en:palm-oil-content-unknown,en:vegan-status-un...",en:none,en:none,1.0,en:e900,NaN,NaN,"en:to-be-completed,en:nutrition-facts-to-be-co...",NaN,NaN,NaN,NaN,fr:lubrifiant-feminin,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

- url --> ok
- created_datetime --> transform to 1 column for year
- last_modified_datetime --> transform to 1 column for year
- product_name --> delete empty rows
- packaging_tags --> set 'en:unknown' for empty rows
- categories_tags --> set 'en:unknown' for empty rows
- ingredients_tags --> delete empty rows
- ingredients_analysis_tags --> set 'unknown' for empty rows
- allergens --> set 'en:none' for empty rows
- traces_tags --> set 'en:none' for empty rows
- additives_n --> ok
- additives_tags --> set en:none when empty
- nutriscore_grade --> set unknown when empty
- food_groups_tags --> set en:none when empty
- states_tags --> ok
- ecoscore_grade --> set unknown when empty
- nutrient_levels_tags --> set en:unknown when empty
- popularity_tags --> set unknown when empty
- main_category --> set en:none when empty
- image_url --> set 'en:none' for empty rows
- energy-kcal_100g --> set -1 for empty rows
- energy_100g --> set -1 for empty rows
- fat_100g --> set -1 for empty rows
- cholesterol_100g --> set -1 for empty rows
- sugars_100g --> set -1 for empty rows
- proteins_100g --> set -1 for empty rows
- salt_100g --> set -1 for empty rows


In [20]:
columns_to_keep = [
    'code',
    'url',
    'last_modified_datetime',
    'product_name',
    'packaging_tags',
    'categories_tags',
    'ingredients_tags',
    'ingredients_analysis_tags',
    'allergens',
    'traces_tags',
    'additives_tags',
    'nutriscore_grade',
    'food_groups_tags',
    'states_tags',
    'ecoscore_grade',
    'nutrient_levels_tags',
    'popularity_tags',
    'main_category',
    'image_url',
    'energy_100g',
    'fat_100g',
    'saturated-fat_100g',
    'cholesterol_100g',
    'sugars_100g',
    'proteins_100g',
    'salt_100g',
    'fruits-vegetables-nuts-estimate-from-ingredients_100g'
]

clean_data = data[columns_to_keep]
display(clean_data)

,code,url,last_modified_datetime,product_name,packaging_tags,categories_tags,ingredients_tags,ingredients_analysis_tags,allergens,traces_tags,additives_tags,nutriscore_grade,food_groups_tags,states_tags,ecoscore_grade,nutrient_levels_tags,popularity_tags,main_category,image_url,energy_100g,fat_100g,saturated-fat_100g,cholesterol_100g,sugars_100g,proteins_100g,salt_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g
4,5,http://world-en.openfoodfacts.org/product/0000...,2024-10-20T14:02:34Z,Bio inulin,en:glass,"en:plant-based-foods-and-beverages,en:plant-ba...","en:apricot-juice-and-puree,en:fruit,en:prunus-...","en:palm-oil-free,en:vegan,en:vegetarian",en:none,en:none,NaN,a,"en:fruits-and-vegetables,en:vegetables","en:to-be-completed,en:nutrition-facts-complete...",unknown,"en:fat-in-low-quantity,en:saturated-fat-in-low...",NaN,en:vegetables,https://images.openfoodfacts.org/images/produc...,840.0,0.0,0.0,NaN,8.0,0.0,0.23,50.000000
9,10,http://world-en.openfoodfacts.org/product/0000...,2024-10-04T09:38:47Z,Madeleines nature,"en:plastic,en:cardboard,fr:boite-en-carton,fr:...","en:snacks,en:desserts,en:sweet-snacks,en:biscu...","en:wheat-flour,en:cereal,en:flour,en:wheat,en:...","en:palm-oil-free,en:non-vegan,en:vegetarian-st...",other,other,"en:e331,en:e422,en:e503",d,"en:sugary-snacks,en:biscuits-and-cakes","en:to-be-completed,en:nutrition-facts-complete...",c,"en:fat-in-high-quantity,en:saturated-fat-in-mo...","top-50000-scans-2019,top-100000-scans-2019,at-...",en:plain-madeleines,https://images.openfoodfacts.org/images/produc...,1852.0,22.0,2.6,NaN,25.0,6.4,0.53,22.666667
14,15,http://world-en.openfoodfacts.org/product/0000...,2024-10-04T09:38:50Z,Madeleines ChocoLait,"en:plastic,en:cardboard","en:snacks,en:sweet-snacks,en:biscuits-and-cake...","en:wheat-flour,en:cereal,en:flour,en:wheat,en:...","en:palm-oil-free,en:non-vegan,en:maybe-vegetarian",en:none,en:nuts,"en:e322,en:e322i,en:e331,en:e422,en:e500,en:e503",d,"en:sugary-snacks,en:biscuits-and-cakes","en:to-be-completed,en:nutrition-facts-complete...",d,"en:fat-in-high-quantity,en:saturated-fat-in-hi...","top-50000-scans-2019,top-100000-scans-2019,at-...",en:chocolate-madeleines,https://images.openfoodfacts.org/images/produc...,1926.0,24.0,6.0,NaN,31.0,6.4,0.48,16.250000
18,20,http://world-en.openfoodfacts.org/product/0000...,2024-10-04T09:38:53Z,Madeleines Choco Noir,fr:1-boite-en-carton-a-recycler-50-sachets-ind...,"en:snacks,en:sweet-snacks,en:biscuits-and-cake...","en:wheat-flour,en:cereal,en:flour,en:wheat,en:...","en:palm-oil-free,en:non-vegan,en:vegetarian-st...",other,en:nuts,"en:e322,en:e331,en:e422,en:e500,en:e503",d,"en:sugary-snacks,en:biscuits-and-cakes","en:to-be-checked,en:complete,en:nutrition-fact...",d,"en:fat-in-high-quantity,en:saturated-fat-in-hi...","top-50000-scans-2019,top-100000-scans-2019,at-...",en:chocolate-madeleines,https://images.openfoodfacts.org/images/produc...,1953.0,25.0,6.1,NaN,29.0,6.3,0.45,16.250000
20,22,http://world-en.openfoodfacts.org/product/0000...,2024-10-04T09:38:54Z,Farandole de madeleine,"fr:boite-en-carton,fr:film-en-plastique","en:snacks,en:sweet-snacks,en:biscuits-and-cake...","fr:madeleines-choconoir,fr:madeleines-nappees-...","en:palm-oil-content-unknown,en:non-vegan,en:ve...",en:none,en:nuts,"en:e322,en:e322i,en:e331,en:e422,en:e500,en:e503",unknown,"en:sugary-snacks,en:biscuits-and-cakes","en:to-be-completed,en:nutrition-facts-complete...",d,NaN,"top-75-percent-scans-2020,top-80-percent-scans...",en:long-madeleines,https://images.openfoodfacts.org/images/produc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3498741,9999864004549,http://world-en.openfoodfacts.org/product/9999...,2019-08-01T11:25:22Z,Boudin à l’ancienne,NaN,"en:meats-and-their-products,en:meats,en:prepar...",fr:biocoop-bordeaux-lac-distribue-par-les-elev...,"en:may-contain-palm-oil,en:non-vegan,en:non-ve...",en:none,en:none,NaN,unknown,"en:fis

In [ ]:
import pandas as pd


def clean_dataset(df):
    # Transformation des dates en année
    df['last_modified_year'] = pd.to_datetime(df['last_modified_datetime']).dt.year
    
    # Supprimer les lignes vides pour certaines colonnes
    df = df.dropna(subset=['product_name', 'ingredients_tags'])
    
    # Remplacer les valeurs vides dans d'autres colonnes
    fill_values = {
        'packaging_tags': 'en:unknown',
        'categories_tags': 'en:unknown',
        'ingredients_analysis_tags': 'unknown',
        'allergens': 'en:none',
        'traces_tags': 'en:none',
        'additives_tags': 'en:none',
        'nutriscore_grade': 'unknown',
        'food_groups_tags': 'en:none',
        'states_tags': 'en:unknown',
        'ecoscore_grade': 'unknown',
        'nutrient_levels_tags': 'en:unknown',
        'popularity_tags': 'unknown',
        'main_category': 'en:none',
        'image_url': 'en:none',
        'energy_100g': -1,
        'fat_100g': -1,
        'saturated-fat_100g': -1,
        'cholesterol_100g': -1,
        'sugars_100g': -1,
        'proteins_100g': -1,
        'salt_100g': -1,
        'fruits-vegetables-nuts-estimate-from-ingredients_100g': -1
    }

    df.fillna(value=fill_values, inplace=True)
    
    # Supprimer les colonnes non nécessaires
    df = df.drop(columns=['last_modified_datetime'])
    
    return df

clean_data = clean_dataset(clean_data)
clean_data.head()


/tmp/ipykernel_198991/172457088.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_198991/172457088.py:36: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,code,url,product_name,packaging_tags,categories_tags,ingredients_tags,ingredients_analysis_tags,allergens,traces_tags,additives_tags,nutriscore_grade,food_groups_tags,states_tags,ecoscore_grade,nutrient_levels_tags,popularity_tags,main_category,image_url,energy_100g,fat_100g,saturated-fat_100g,cholesterol_100g,sugars_100g,proteins_100g,salt_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g,last_modified_year
4,5,http://world-en.openfoodfacts.org/product/0000...,Bio inulin,en:glass,"en:plant-based-foods-and-beverages,en:plant-ba...","en:apricot-juice-and-puree,en:fruit,en:prunus-...","en:palm-oil-free,en:vegan,en:vegetarian",en:none,en:none,en:none,a,"en:fruits-and-vegetables,en:vegetables","en:to-be-completed,en:nutrition-facts-complete...",unknown,"en:fat-in-low-quantity,en:saturated-fat-in-low...",unknown,en:vegetables,https://images.openfoodfacts.org/images/produc...,840.0,0.0,0.0,-1.0,8.0,0.0,0.23,50.000000,2024
9,10,http://world-en.openfoodfacts.org/product/0000...,Madeleines nature,"en:plastic,en:cardboard,fr:boite-en-carton,fr:...","en:snacks,en:desserts,en:sweet-snacks,en:biscu...","en:wheat-flour,en:cereal,en:flour,en:wheat,en:...","en:palm-oil-free,en:non-vegan,en:vegetarian-st...",other,other,"en:e331,en:e422,en:e503",d,"en:sugary-snacks,en:biscuits-and-cakes","en:to-be-completed,en:nutrition-facts-complete...",c,"en:fat-in-high-quantity,en:saturated-fat-in-mo...","top-50000-scans-2019,top-100000-scans-2019,at-...",en:plain-madeleines,https://images.openfoodfacts.org/images/produc...,1852.0,22.0,2.6,-1.0,25.0,6.4,0.53,22.666667,2024
14,15,http://world-en.openfoodfacts.org/product/0000...,Madeleines ChocoLait,"en:plastic,en:cardboard","en:snacks,en:sweet-snacks,en:biscuits-and-cake...","en:wheat-flour,en:cereal,en:flour,en:wheat,en:...","en:palm-oil-free,en:non-vegan,en:maybe-vegetarian",en:none,en:nuts,"en:e322,en:e322i,en:e331,en:e422,en:e500,en:e503",d,"en:sugary-snacks,en:biscuits-and-cakes","en:to-be-completed,en:nutrition-facts-complete...",d,"en:fat-in-high-quantity,en:saturated-fat-in-hi...","top-50000-scans-2019,top-100000-scans-2019,at-...",en:chocolate-madeleines,https://images.openfoodfacts.org/images/produc...,1926.0,24.0,6.0,-1.0,31.0,6.4,0.48,16.250000,2024
18,20,http://world-en.openfoodfacts.org/product/0000...,Madeleines Choco Noir,fr:1-boite-en-carton-a-recycler-50-sachets-ind...,"en:snacks,en:sweet-snacks,en:biscuits-and-cake...","en:wheat-flour,en:cereal,en:flour,en:wheat,en:...","en:palm-oil-free,en:non-vegan,en:vegetarian-st...",other,en:nuts,"en:e322,en:e331,en:e422,en:e500,en:e503",d,"en:sugary-snacks,en:biscuits-and-cakes","en:to-be-checked,en:complete,en:nutrition-fact...",d,"en:fat-in-high-quantity,en:saturated-fat-in-hi...","top-50000-scans-2019,top-100000-scans-2019,at-...",en:chocolate-madeleines,https://images.openfoodfacts.org/images/produc...,1953.0,25.0,6.1,-1.0,29.0,6.3,0.45,16.250000,2024
20,22,http://world-en.openfoodfacts.org/product/0000...,Farandole de madeleine,"fr:boite-en-carton,fr:film-en-plastique","en:snacks,en:sweet-snacks,en:biscuits-and-cake...","fr:madeleines-choconoir,fr:madeleines-nappees-...","en:palm-oil-content-unknown,en:non-vegan,en:ve...",en:none,en:nuts,"en:e322,en:e322i,en:e331,en:e422,en:e500,en:e503",unknown,"en:sugary-snacks,en:biscuits-and-cakes","en:to-be-completed,en:nutrition-facts-complete...",d,en:unknown,"top-75-percent-scans-2020,top-80-percent-scans...",en:long-madeleines,https://images.openfoodfacts.org/images/produc...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.00,1.750000,2024


In [22]:
dftest= clean_data['main_category'].value_counts().reset_index()
dftest[dftest['count'] > 100]

,main_category,count
0,en:none,34985
1,en:groceries,8649
2,en:sweetened-beverages,4210
3,en:beverages,1725
4,en:candies,1474
...,...,...
427,en:smoked-sausages,102
428,en:canned-flageolet-beans,102
429,en:sardines-in-tomato-sauce,101
430,en:blackberry-jams,101


In [23]:
clean_data.describe(include='all')

,code,url,product_name,packaging_tags,categories_tags,ingredients_tags,ingredients_analysis_tags,allergens,traces_tags,additives_tags,nutriscore_grade,food_groups_tags,states_tags,ecoscore_grade,nutrient_levels_tags,popularity_tags,main_category,image_url,energy_100g,fat_100g,saturated-fat_100g,cholesterol_100g,sugars_100g,proteins_100g,salt_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g,last_modified_year
count,2.555370e+05,255537,255537,255537,255537,255537,255537,255537,255537,255537,255537,255537,255537,255537,255537,255537,255537,255537,2.555370e+05,255537.000000,255537.000000,255537.000000,255537.000000,255537.000000,255537.000000,255537.000000,255537.000000
unique,2.547650e+05,254771,197147,18379,49352,194509,38,19,33,32825,7,46,1653,9,224,85393,16911,237020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,3.254560e+12,http://world-en.openfoodfacts.org/product/3433...,Comté,en:unknown,en:unknown,"en:superior-quality-durum-wheat-semolina,en:ce...","en:palm-oil-free,en:vegan,en:vegetarian",en:none,en:none,en:none,unknown,en:none,"en:to-be-completed,en:nutrition-facts-complete...",unknown,en:unknown,unknown,en:none,en:none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,2.000000e+00,2,204,125404,34985,784,45120,156416,179620,124985,54476,56622,20867,92392,62937,72025,34985,17773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.328430e+11,12.047864,4.571277,-0.997241,12.075284,7.374202,0.896229,19.172789,2022.430744
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.177037e+14,16.715739,7.752517,0.091233,19.215145,9.121442,6.788525,48.506079,1.454487
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.000000e+00,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-65.887256,2013.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.930000e+02,0.500000,0.100000,-1.000000,0.500000,0.600000,0.010000,0.000000,2022.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.080000e+02,5.100000,1.300000,-1.000000,2.700000,5.200000,0.275000,0.056676,2023.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.570000e+03,19.000000,6.000000,-1.000000,15.000000,10.770000,1.100000,22.833333,2023.000000


In [24]:
clean_data.to_csv("clean_dataset.csv")

### Text processing

In [25]:
clean_data = pd.read_csv("clean_dataset.csv")

/tmp/ipykernel_198991/3843401428.py:1: DtypeWarning:

Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.



In [46]:
nlp = en_core_web_sm.load()

text_processing = TextProcessing()

def clean_text_column(text):
    # Suppression des préfixes de langue
    text = re.sub(r'\b\w{2}:\b', ' ', text)  # \b délimite le mot et \w{2} correspond à deux lettres
    # Remplacement des tirets par des espaces
    text = text.replace('-', ' ')
    # Suppression des espaces multiples
    text = re.sub(r'\s+', ' ', text).strip()
    return text

fill_values = {
        'packaging_tags': 'en:unknown',
        'categories_tags': 'en:unknown',
        'ingredients_analysis_tags': 'unknown',
        'allergens': 'en:none',
        'traces_tags': 'en:none',
        'additives_tags': 'en:none',
        'nutriscore_grade': 'unknown',
        'food_groups_tags': 'en:none',
        'states_tags': 'en:unknown',
        'ecoscore_grade': 'unknown',
        'nutrient_levels_tags': 'en:unknown',
        'popularity_tags': 'unknown',
        'main_category': 'en:none',
        'image_url': 'en:none',
        'energy_100g': -1,
        'fat_100g': -1,
        'saturated-fat_100g': -1,
        'cholesterol_100g': -1,
        'sugars_100g': -1,
        'proteins_100g': -1,
        'salt_100g': -1,
        'fruits-vegetables-nuts-estimate-from-ingredients_100g': -1
    }

# Liste des colonnes à traiter
columns_cat = ['product_name', 'packaging_tags', 'categories_tags', 'ingredients_tags', 'ingredients_analysis_tags', 
           'nutrient_levels_tags', 'nutriscore_grade', 'main_category', 'ecoscore_grade', 'popularity_tags']

# Liste pour stocker les noms des nouvelles colonnes créées
new_columns_cat_names = []

# Boucle pour appliquer les transformations et stocker les noms des nouvelles colonnes
for col in columns_cat:
    # Nom des nouvelles colonnes pour chaque transformation
    preprocessed_col = f'preprocessed_{col}'
    # lemmatized_col = f'preprocessed_{col}_lemmatized'
    len_col = f'len_{preprocessed_col}'
    
    # Appliquer le nettoyage
    clean_data[preprocessed_col] = clean_data[col].fillna('').apply(clean_text_column)
    
    # Appliquer la standardisation
    clean_data[preprocessed_col] = clean_data[preprocessed_col].apply(text_processing.standardize)

    # Calculer la longueur des mots dans la colonne lemmatisée
    clean_data[len_col] = clean_data[preprocessed_col].apply(lambda x: len(x))
    
    # Appliquer la lemmatisation
    # clean_data[lemmatized_col] = clean_data[preprocessed_col].apply(lambda x: text_processing.lemmatize(x, nlp))
    
    # Calculer la longueur des mots dans la colonne lemmatisée
    # clean_data[len_col] = clean_data[lemmatized_col].apply(lambda x: len(x))
    
    # Ajouter les nouveaux noms de colonnes dans la liste
    # new_columns_cat_names.extend([preprocessed_col, lemmatized_col, len_col])
    new_columns_cat_names.extend([preprocessed_col, len_col])

# Affichage des noms des nouvelles colonnes créées
print("Noms des nouvelles colonnes créées :")
print(new_columns_cat_names)
# Affichage des premières lignes des nouvelles colonnes
clean_data[new_columns_cat_names].head()

Noms des nouvelles colonnes créées :
['preprocessed_product_name', 'len_preprocessed_product_name', 'preprocessed_packaging_tags', 'len_preprocessed_packaging_tags', 'preprocessed_categories_tags', 'len_preprocessed_categories_tags', 'preprocessed_ingredients_tags', 'len_preprocessed_ingredients_tags', 'preprocessed_ingredients_analysis_tags', 'len_preprocessed_ingredients_analysis_tags', 'preprocessed_nutrient_levels_tags', 'len_preprocessed_nutrient_levels_tags', 'preprocessed_nutriscore_grade', 'len_preprocessed_nutriscore_grade', 'preprocessed_main_category', 'len_preprocessed_main_category', 'preprocessed_ecoscore_grade', 'len_preprocessed_ecoscore_grade', 'preprocessed_popularity_tags', 'len_preprocessed_popularity_tags']


,preprocessed_product_name,len_preprocessed_product_name,preprocessed_packaging_tags,len_preprocessed_packaging_tags,preprocessed_categories_tags,len_preprocessed_categories_tags,preprocessed_ingredients_tags,len_preprocessed_ingredients_tags,preprocessed_ingredients_analysis_tags,len_preprocessed_ingredients_analysis_tags,preprocessed_nutrient_levels_tags,len_preprocessed_nutrient_levels_tags,preprocessed_nutriscore_grade,len_preprocessed_nutriscore_grade,preprocessed_main_category,len_preprocessed_main_category,preprocessed_ecoscore_grade,len_preprocessed_ecoscore_grade,preprocessed_popularity_tags,len_preprocessed_popularity_tags
0,bio inulin,10,glass,5,plant based foods and beverages plant based fo...,117,apricot juice and puree fruit prunus species f...,123,palm oil free vegan vegetarian,30,fat in low quantity saturated fat in low quant...,98,a,1,vegetables,10,unknown,7,unknown,7
1,madeleines nature,17,plastic cardboard boite en carton film en plas...,51,snacks desserts sweet snacks biscuits and cake...,81,wheat flour cereal flour wheat cereal flour su...,435,palm oil free non vegan vegetarian status unknown,49,fat in high quantity saturated fat in moderate...,105,d,1,plain madeleines,16,c,1,top 50000 scans 2019top 100000 scans 2019at le...,1572
2,madeleines chocolait,20,plastic cardboard,17,snacks sweet snacks biscuits and cakes cakes c...,92,wheat flour cereal flour wheat cereal flour mi...,564,palm oil free non vegan maybe vegetarian,40,fat in high quantity saturated fat in high qua...,101,d,1,chocolate madeleines,20,d,1,top 50000 scans 2019top 100000 scans 2019at le...,1641
3,madeleines choco noir,21,1 boite en carton a recycler 50 sachets indivi...,62,snacks sweet snacks biscuits and cakes cakes c...,92,wheat flour cereal flour wheat cereal flour da...,557,palm oil free non vegan vegetarian status unknown,49,fat in high quantity saturated fat in high qua...,101,d,1,chocolate madeleines,20,d,1,top 50000 scans 2019top 100000 scans 2019at le...,1571
4,farandole de madeleine,22,boite en carton film en plastique,33,snacks sweet snacks biscuits and cakes cakes c...,108,madeleines choconoir madeleines nappees de cho...,951,palm oil content unknown non vegan vegetarian ...,60,unknown,7,unknown,7,long madeleines,15,d,1,top 75 percent scans 2020top 80 percent scans ...,603


## Training

Essai avec un entrainement d'un modèle simple Kmeans.

Rappel :

**1. CountVectorizer**

- **Principe :** Compte le nombre d’occurrences de chaque mot dans un texte ou une liste de tags.
- **Utilisation :** Représente chaque mot par une colonne dans une matrice, avec une valeur représentant sa fréquence.
- **Avantage :** Simple et rapide à calculer, utile pour des listes ou textes courts où la présence des mots est suffisante.
- **Limite :** Ne capture pas les relations sémantiques (ex. "sucre" ≠ "glucose").

**2. TF-IDF Vectorizer**

- **Principe :** Calcule la fréquence des mots ajustée par leur rareté dans l’ensemble des documents.
- **Utilisation :** Les mots fréquents dans peu de documents reçoivent une valeur plus élevée, permettant de distinguer des termes spécifiques.
- **Avantage :** Diminue l’importance des mots très fréquents (ex. "de", "le") et met en avant les mots distinctifs.
- **Limite :** Ne capture pas la similarité sémantique entre les mots.

**3. Word Embeddings (Word2Vec, GloVe)**

- **Principe :** Apprend à représenter chaque mot dans un espace vectoriel de manière à ce que des mots contextuellement proches soient également proches dans l’espace vectoriel.
- **Utilisation :** Représente chaque mot par un vecteur de plusieurs dimensions, capturant des relations sémantiques (ex. "roi" proche de "reine").
- **Avantage :** Capture les similarités et relations entre mots, utile pour les tâches nécessitant une compréhension sémantique.
- **Limite :** Ne prend pas en compte le contexte spécifique d'une phrase.

**4. FastText**

- **Principe :** Apprend également des embeddings mais utilise des sous-mots (n-grammes de caractères), ce qui permet de mieux comprendre les mots avec des similitudes orthographiques.
- **Exemple :** FastText repère des similarités entre "glucose" et "sucre" en utilisant des sous-parties communes dans les mots. Si des variantes de mots sont dans le dataset (ex. "sucreries", "sucre"), FastText peut capturer ces relations plus finement.
- **Avantage :** Idéal pour des données contenant des variantes ou des erreurs typographiques, tout en offrant des similarités proches de Word2Vec et BERT pour les termes proches.
- **Limite :** FastText est limité par son incapacité à capter le contexte global, la syntaxe et la structure grammaticale, ce qui le rend moins performant pour comprendre le sens des mots dans des phrases complexes ou ambiguës

**5. BERT (Contextual Embeddings)**

- **Principe :** Génère des représentations contextuelles pour chaque mot dans une phrase, tenant compte du contexte global.
- **Utilisation :** Représente chaque mot différemment selon le contexte de la phrase (ex. "batterie" dans "batterie de cuisine" vs. "batterie de voiture").
- **Avantage :** Idéal pour les phrases et textes complexes nécessitant une compréhension fine du contexte.
- **Limite :** Modèle très coûteux en calcul, moins utile pour des listes simples de mots isolés.

--- 

**En résumé :**

CountVectorizer et TF-IDF sont légers et efficaces pour des listes de tags ou des mots courts.

Word Embeddings ajoutent la dimension sémantique.

BERT excelle dans des phrases contextuelles, mais est plus lourd et parfois excessif pour des mots isolés.

In [ ]:
# Colonnes numériques et catégorielles
numeric_columns = [
    'energy_100g', 'fat_100g', 'saturated-fat_100g', 'cholesterol_100g', 
    'sugars_100g', 'proteins_100g', 'salt_100g', 'fruits-vegetables-nuts-estimate-from-ingredients_100g'
]
categorial_columns = [
    'preprocessed_product_name', 'preprocessed_packaging_tags', 'preprocessed_categories_tags', 
    'preprocessed_ingredients_tags', 'preprocessed_ingredients_analysis_tags', 
    'preprocessed_nutriscore_grade', 'preprocessed_main_category'
]

# Fonction d'imputation pour les valeurs numériques
def impute_numeric_data(df, columns):
    imputer = SimpleImputer(strategy='mean')
    df[columns] = imputer.fit_transform(df[columns])
    return df

# Fonction de normalisation des colonnes numériques
def scale_numeric_data(df, columns):
    scaler = StandardScaler()
    df[columns] = scaler.fit_transform(df[columns])
    return df

# Fonction de transformation de texte pour les colonnes catégorielles
def encode_categorical_data(df, columns, max_features=500, min_df=4, n_components=50):
    vectorizer = CountVectorizer(max_features=max_features, min_df=min_df)
    encoded_columns = []
    
    for col in columns:
        print('column', col)
        # Encodage du texte
        encoded = vectorizer.fit_transform(df[col].fillna(''))
        print('encoded.shape', encoded.shape[1])
        
        # Ajustement de n_components pour TruncatedSVD si nécessaire
        adjusted_n_components = min(n_components, encoded.shape[1])
        print('adjusted_n_components:', adjusted_n_components)
        # Réduction de dimensions avec TruncatedSVD plus optimisé avec les sparses matrices que PCA
        svd = TruncatedSVD(n_components=adjusted_n_components, random_state=42)
        
        # Réduction de dimensions
        reduced = svd.fit_transform(encoded)
        encoded_columns.append(pd.DataFrame(reduced))
    
    vectorizer_nb_features = vectorizer.get_feature_names_out()
    print("[INFO][CountVectorizer] : Nombre de features dans le vocabulaire : ", len(vectorizer_nb_features))
    return pd.concat(encoded_columns, axis=1)

# Fonction de clustering et calcul des scores
def cluster_data(features, cluster_range, random_state=42):
    silhouette_scores = []
    inertia = []
    
    for k in cluster_range:
        kmeans = KMeans(n_clusters=k, random_state=random_state)
        labels = kmeans.fit_predict(features)
        inertia.append(kmeans.inertia_)
        silhouette_scores.append(silhouette_score(features, labels))
    
    return labels, silhouette_scores, inertia

# Fonction principale de préparation des données et de clustering
def prepare_and_cluster(df, cluster_range, random_state=42):
    # Imputation des valeurs numériques
    df = impute_numeric_data(df, numeric_columns)
    
    # Encodage des données catégorielles
    categorical_features = encode_categorical_data(df, categorial_columns)
    
    # Normalisation des données numériques
    df = scale_numeric_data(df, numeric_columns)
    
    # Assemblage des features
    features = pd.concat([categorical_features, df[numeric_columns].reset_index(drop=True)], axis=1)
    features.columns = features.columns.astype(str)
    
    # Clustering et calcul des scores
    labels, silhouette_scores, inertia = cluster_data(features, cluster_range=cluster_range, random_state=random_state)
    
    # Ajout des labels de clusters au DataFrame
    df['cluster'] = labels
    return df, silhouette_scores, inertia

def train_kmeans_with_manual_clusters(df, n_clusters, random_state=42):
    """
    Applique un KMeans avec un nombre de clusters spécifié manuellement.
    
    Parameters:
        df (DataFrame): Le DataFrame contenant les données sur lesquelles appliquer le KMeans.
        n_clusters (int): Le nombre de clusters à utiliser.
        random_state (int): L'état aléatoire pour assurer la reproductibilité.
    
    Returns:
        DataFrame: Le DataFrame avec les labels de clusters ajoutés.
    """
    # Imputation des valeurs numériques
    df = impute_numeric_data(df, numeric_columns)
    
    # Encodage des données catégorielles
    categorical_features = encode_categorical_data(df, categorial_columns)
    
    # Normalisation des données numériques
    df = scale_numeric_data(df, numeric_columns)
    
    # Assemblage des features
    features = pd.concat([categorical_features, df[numeric_columns].reset_index(drop=True)], axis=1)
    features.columns = features.columns.astype(str)
    
    # Apprentissage du modèle KMeans avec le nombre de clusters spécifié
    kmeans = KMeans(n_clusters=n_clusters, random_state=random_state)
    df['cluster'] = kmeans.fit_predict(features)
    
    return df


def inertia_elbow_plotly(cluster_range, inertia, silhouette_scores):
    data = pd.DataFrame({
        'Number of clusters': cluster_range,
        'Inertia': inertia,
        'Silhouette Score': silhouette_scores
    })
    
    # Graphique Elbow (Inertia)
    fig_inertia = px.line(
        data,
        x='Number of clusters',
        y='Inertia',
        markers=True,
        title="Elbow Method"
    )
    fig_inertia.update_layout(
        xaxis_title="Number of clusters",
        yaxis_title="Inertia"
    )
    
    # Graphique Silhouette
    fig_silhouette = px.line(
        data,
        x='Number of clusters',
        y='Silhouette Score',
        markers=True,
        title="Silhouette Score"
    )
    fig_silhouette.update_layout(
        xaxis_title="Number of clusters",
        yaxis_title="Silhouette Score"
    )
    
    fig_inertia.show()
    fig_silhouette.show()


cluster_range=range(50, 1000, 25)

df_kmeans, silhouette_scores, inertia = prepare_and_cluster(df=clean_data.sample(2000), cluster_range=cluster_range)

# Affichage des résultats
df_kmeans.head()
print("Silhouette Scores:", silhouette_scores)
print("Inertia:", inertia)

inertia_elbow_plotly(cluster_range=cluster_range, inertia=inertia, silhouette_scores=silhouette_scores)


column preprocessed_product_name
encoded.shape 210
adjusted_n_components: 50
column preprocessed_packaging_tags
encoded.shape 74
adjusted_n_components: 50
column preprocessed_categories_tags
encoded.shape 210
adjusted_n_components: 50
column preprocessed_ingredients_tags
encoded.shape 210
adjusted_n_components: 50
column preprocessed_ingredients_analysis_tags
encoded.shape 12
adjusted_n_components: 12
column preprocessed_nutriscore_grade
encoded.shape 3
adjusted_n_components: 3
column preprocessed_main_category
encoded.shape 210
adjusted_n_components: 50
[INFO][CountVectorizer] : Nombre de features dans le vocabulaire :  210
Silhouette Scores: [0.1005999983899091, 0.10768130300298916, 0.10812062844501151, 0.09925294562462046, 0.10564067098305037, 0.10282925076192623, 0.10037636686015056, 0.10149909564400263, 0.10002394028950201, 0.10038431164257546, 0.10279484775593742, 0.10494296374715295, 0.10605293312265436, 0.10945479785378254, 0.10927545292749946, 0.10933242903762484, 0.1108142818

In [32]:
 # Appliquer KMeans avec le meilleur nombre de clusters
df_result= train_kmeans_with_manual_clusters(df=clean_data, n_clusters=625)

column preprocessed_product_name
encoded.shape 210
adjusted_n_components: 50
column preprocessed_packaging_tags
encoded.shape 210
adjusted_n_components: 50
column preprocessed_categories_tags
encoded.shape 210
adjusted_n_components: 50
column preprocessed_ingredients_tags
encoded.shape 210
adjusted_n_components: 50
column preprocessed_ingredients_analysis_tags
encoded.shape 12
adjusted_n_components: 12
column preprocessed_nutriscore_grade
encoded.shape 3
adjusted_n_components: 3
column preprocessed_main_category
encoded.shape 210
adjusted_n_components: 50
[INFO][CountVectorizer] : Nombre de features dans le vocabulaire :  210


In [33]:
df_result.shape

(255537, 49)

In [34]:
sample_df = df_result.sample(1000)
sample_df.to_csv("sample.csv")

In [35]:
cluster_count_df =df_result["cluster"].value_counts().reset_index()
cluster_count_df

,cluster,count
0,44,4132
1,548,2820
2,132,1719
3,32,1684
4,310,1584
...,...,...
620,19,1
621,21,1
622,82,1
623,97,1


In [36]:
cluster_count_df[cluster_count_df["count"] > 10]

,cluster,count
0,44,4132
1,548,2820
2,132,1719
3,32,1684
4,310,1584
...,...,...
612,476,55
613,263,47
614,127,45
615,497,41


In [37]:
df_result["cluster"].value_counts()


df_result[["product_name"]][df_result["cluster"] == 100].head(20)

,product_name
4682,Pate croute richelieu canard
4709,Roti de veau à l'indienne
5722,10 Saucisses de Strasbourg
5727,Mousse de foie forestière
6985,Metka Petite
13962,Prinskorv - Saucisses de porc et de bœuf
14213,Cervelas de Strasbourg
14406,Knack filière
16261,Saucisse reunionnaise fumee
16362,Saucisse fumée réunionnaise


In [38]:
df_result[["product_name"]][df_result["cluster"] == 20].head(20)

,product_name
6803,Nespresso Origin Brazil
46590,Café du Pérou bio et équitable capsules x10
46591,Classico lungo
46595,Capsules de café expresso
61355,Forza café Espresso
70254,Espresso Forte
77350,Café expresso classico
77351,Café espresso fortissimo
77352,Café espresso decaffeinato
77355,Café espresso leggero


In [39]:
df_result[["product_name"]][df_result["cluster"] == 169].head(20)

,product_name
166,Ice cream sticks Vanilla
862,macadamia but brittle
2856,New York Super Fudge Chunk Ice Cream
2858,Ben & Jerry's Dessert Glacé MENTHE CHOCOLAT 45...
9395,Adelis glace
11333,Favorite selection mini cups
15503,Glace NOUGAT de Montélimar Chabert & Guillot
15504,Glace RÉGLISSE Intense
15505,"Glace RHUM RAISINS, raisins gloden macérés"
15507,Glace à la VERVEINE DU VELAY


In [40]:
df_result[["product_name"]][df_result["cluster"] == 165].head(20)

,product_name
10662,Sauce bourguignonne
11199,Sauce Pommes Frites
14977,Dips Sauce Creamy Curry
14996,Sauce Creamy Deluxe
15099,Sauce Tartare
15514,Sauce Curry
19506,Sauce bourguignonne
19508,Sauce Tartare
20190,Sauce Pommes Frites
20911,Sauce crudités caesar
